In [5]:
DATAPATH = "/mnt/cloud/lithuanian-asr/data"

In [1]:
!pip install pympi-ling

In [2]:
import pympi
from pathlib import Path
from pydub import AudioSegment

In [5]:
def get_eaf_data(filename):
    filepath = Path(filename)
    base = filepath.stem
    eaf = pympi.Elan.Eaf(filename)
    tiers = []

    for tier in eaf.tiers['speech'][0].keys():
        current = {}
        id = f"{base}_{tier}"
        current["id"] = id
        data = eaf.tiers['speech'][0][tier]
        current["start"] = eaf.timeslots[data[0]]
        current["end"] = eaf.timeslots[data[1]]
        current["text"] = data[2]
        tiers.append(current)

    return tiers

In [3]:
def write_split_wavs(outdir, filename, data):
    outpath = Path(outdir)
    original = AudioSegment.from_wav(filename)
    for piece in data:
        outfile = outpath / f"{piece['id']}.wav"
        audio = original[piece["start"]:piece["end"]]
        audio.export(str(outfile), format="wav")


In [6]:
SAMPLE = "/mnt/cloud/lithuanian-asr/data/R_RS_F3_AS113_01.eaf"
SAMPLE_WAV = SAMPLE.replace(".eaf", ".wav")
data = get_eaf_data(SAMPLE)
write_split_wavs("/tmp/foo", SAMPLE_WAV, data)

In [6]:
for eaf_file in Path(DATAPATH).glob("*.eaf"):
    print(eaf_file)

/mnt/cloud/lithuanian-asr/data/R_RS_F4_VK112_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_F3_VP468_03.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_F3_VK040_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_F4_VP154_02.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_M4_DK002_02.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_F4_ES147_02.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_F3_VP159_02.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_M4_VP422_01.eaf
/mnt/cloud/lithuanian-asr/data/L_ST_M3_BG190_03.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_M3_AS199_01.eaf
/mnt/cloud/lithuanian-asr/data/L_SD_F4_MS032_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_M4_IS050_01.eaf
/mnt/cloud/lithuanian-asr/data/X_MS001_01.eaf
/mnt/cloud/lithuanian-asr/data/L_RS_F4_MS003_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_F4_BG171_02.eaf
/mnt/cloud/lithuanian-asr/data/R_RT_F4_IS128_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RD_M3_AS185_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_M4_IS063_01.eaf
/mnt/cloud/lithuanian-asr/data/R_RS_F1_IS282_01.eaf
/mnt/cloud/lithuan